## Data Wrangling with Pandas

**[Pandas](http://pandas.pydata.org)** is a Python library that provides extensive means for data analysis. Data scientists often work with data stored in table formats like `.csv`, `.tsv`, or `.xlsx`. Pandas makes it very convenient to load, process, and analyze such tabular data using SQL-like queries. In conjunction with `Matplotlib` and `Seaborn`, `Pandas` provides a wide range of opportunities for visual analysis of tabular data. 

The main data structures in `Pandas` are implemented with **Series** and **DataFrame** classes. The former is a one-dimensional indexed array of some fixed data type. The latter is a two-dimensional data structure - a table - where each column contains data of the same type. You can see it as a dictionary of `Series` instances. `DataFrames` are great for representing real data: rows correspond to instances (examples, observations, etc.), and columns correspond to features of these instances.

# Book 
# http://bit.ly/2RJCXx3

![](../img/books.png)

** Let's get the data **

In [ ]:
import pandas as pd

We’ll demonstrate the main methods in action by analyzing a sample dataset provided by PGGM. Let’s read the data (using `read_csv`), and take a look at the first 5 lines using the `head` method:

In [ ]:
data = pd.read_csv('http://bit.ly/PGGM_dataset')

** Shape of the data **

In [ ]:
data.head()

In [ ]:
data.describe()

In [ ]:
data.columns

In [ ]:
data.info()

In order to see statistics on non-numerical features, one has to explicitly indicate data types of interest in the `include` parameter.

In [ ]:
data.describe(include=['object'])

For categorical (type `object`) and boolean (type `bool`) features we can use the `value_counts` method. Let’s have a look at the distribution of `Sector`:

To calculate fractions, pass `normalize=True` to the `value_counts` function.

In [ ]:
data['GICS_Sector'].value_counts(normalize=True)

### Sorting

A DataFrame can be sorted by the value of one of the variables (i.e columns). For example, we can sort by values of a variable (use ascending=False to sort in descending order):

In [ ]:
data.sort_values(by='Price_USD', ascending=False)

### Missing values

In [ ]:
mis_val = data.isnull().sum()
mis_val_percent = 100 * data.isnull().sum()/len(data)
mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)

In [ ]:
data.isnull().sum()

**Direct replacement**

In [ ]:
data[data.Universe_Returns_F1W.isnull()]['Universe_Returns_F1W']

In [ ]:
data.loc[[2862,19657,35229,35411],'Universe_Returns_F1W'] = 0

**Loop replacement**

In [ ]:
indexes_to_check = data[data.Universe_Returns_F4W.isnull()].index

In [ ]:
for i in indexes_to_check:
    data.loc[i,'Universe_Returns_F4W'] = 0

**Just Fill in**

In [ ]:
data.Universe_Returns_F12W.fillna(value=1, inplace=True)

**Also with operations**

In [ ]:
NTM_EP_mean = data.NTM_EP.mean()

In [ ]:
data.NTM_EP.fillna(value=NTM_EP_mean, inplace=True)

**Or just don't take the na events into account**

In [ ]:
data.dropna(inplace=True)

### Data update

Let's suppose we don't need the commas

In [ ]:
data.Name.loc[7].replace(',','')

In [ ]:
data.loc[7,'Name'] = data.Name.loc[7].replace(',','')

In [ ]:
data.loc[7,'Name']

In [ ]:
for i in data.index:
    data.loc[i,'Name'] = data.Name.loc[i].replace(',','')

### Handling Datetime

In [ ]:
data.Period_YYYYMMDD

In [ ]:
[len(str(i)) for i in data.Period_YYYYMMDD]

In [ ]:
data.Period_YYYYMMDD.map(str).map(len).unique()

** Updateting via list comprehension** 

In [ ]:
data.Period = [pd.to_datetime(i, format='%Y%m%d') for i in data.Period_YYYYMMDD]

** Creating periods **

In [ ]:
data['month'] = [date.month for date in data.Period]

In [ ]:
data['day'] = [date.day for date in data.Period]

In [ ]:
data['year'] = [date.year for date in data.Period]

In [ ]:
data.Period.describe()

### Data Transformations

Supose it is needed a subset of the dataset

In [ ]:
data.GICS_Sector.unique()

** Slicing with logical operations **

In [ ]:
IT_2018_df = data[(data.GICS_Sector == 'Information Technology') & (data.year == 2018)]

** Columns selection **

In [ ]:
IT_2018_df.columns

** Explicit**

In [ ]:
IT_2018_df_ = IT_2018_df[['Ticker','Name','Period','Price_USD']]

** Range **

In [ ]:
IT_2018_df[IT_2018_df.columns[2:6]].head()

** Exception **

In [ ]:
IT_2018_df_.iloc[:,~IT_2018_df_.columns.isin(['index','Period_YYYYMMDD','month','day','year'])].head()

** Pivoting **

In [ ]:
summary_IT = pd.pivot_table(IT_2018_df_, index='Name', columns='Period')

** Transposing **

In [ ]:
transposed = summary_IT.T

In [ ]:
transposed.head()

## Write to file

In [ ]:
transposed.to_csv('name_of_file.csv')

In [ ]:
!rm name_of_file.csv

## Adding Features

We can always extend the knowledge of the dataset, spliting apart or creating new variables

** Labeling **

In [ ]:
market_label = []
for i in data.Market_Cap_USD:
    if i >= 38000:
        market_label.append('>38')
    elif (i < 38000) | (i > 20000):
        market_label.append('<28 & >20')
    else:
        market_label.append('<20')

In [ ]:
data['market_label'] = market_label

** Opeations **

In [ ]:
data['LTM_AVG'] = (data.LTM_ROA + data.LTM_EP) /2

** Fixed values **

In [ ]:
data['extra'] = ['extra'] * len(data)

** Aggregations **

In [ ]:
year_distr = data.groupby('year').count()['Identifier'].reset_index().sort_values('year')

In [ ]:
year_distr['relative'] = year_distr.Identifier / year_distr.Identifier.sum() *100

** Sorting **

In [ ]:
data.groupby('Identifier').count()['Name'].sort_values(ascending=False).head()

** Merging **

In [ ]:
df_Y2573F10 = data[data.Identifier == 'Y2573F10']

In [ ]:
df_Y2573F10 = df_Y2573F10[['Period','Market_Cap_USD','Name']]

In [ ]:
df_14365830 = data[data.Identifier == '14365830']

In [ ]:
df_14365830 = df_14365830[['Period','Market_Cap_USD','Name']]

In [ ]:
portfolio_two = pd.merge(df_Y2573F10, df_14365830, on='Period', how='inner')

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
portfolio_two._get_numeric_data().plot()

### Writing files

In [ ]:
excelbook = pd.ExcelWriter('output.xlsx')

In [ ]:
portfolio_two.to_excel(excelbook, 'sheet1')
portfolio_two.to_excel(excelbook, 'sheet2')
excelbook.save()

Pandas, overall, is one of the reason why Python is such a great software. 
There are many other interesting pandas features I could have shown, but it’s already enough to understand why a data scientist cannot do without pandas. To sum up, pandas is simple to use, hiding all the complex and abstract computations behind (generally) intuitive fast, if not the fastest data analysis package (it highly optimized in C) It is THE tool that helps a data scientist to quickly read and understand data and be more efficient at his role.

### Data Exploration

Data Dimension Milestones

- Data shape
- Datasets Treatment  
- Variables Analysis  
- Descriptive Statistics  

In [ ]:
tuples = []
for column in data.columns:
    tuples.append((column, data[str(column)].map(type).unique()))

In [ ]:
year_distr =data.groupby('year').count()['Identifier'].reset_index().sort_values('year')

In [ ]:
year_distr['relative'] = year_distr.Identifier / year_distr.Identifier.sum() *100

** Uniqueness **

In [ ]:
sector = data.GICS_Sector.unique()

In [ ]:
print(len(sector), sector)

In [ ]:
plt.barh(df_sector.GICS_Sector, df_sector.relative)

#### Now with the other category

In [ ]:
df_ind_grp = data.groupby('GICS_Ind_Grp').count()['Identifier'].reset_index().sort_values(by='Identifier')

In [ ]:
plt.figure(figsize=(10,7))
plt.barh(df_ind_grp.GICS_Ind_Grp,df_ind_grp.Identifier)

#### Now we go with the continous variables

In [ ]:
df_assets.Price_USD.mean()

In [ ]:
df_assets.Price_USD.std()

In [ ]:
df_assets._get_numeric_data().describe().T

Let's take the market capital and plot it over the time

In [ ]:
df_assets.groupby('Period').mean()['Market_Cap_USD'].plot()